In [1]:
import pandas as pd
from data_transform import DataTransform
from data_frame_info import DataFrameInfo
from plotter import Plotter
from data_frame_transform import DataFrameTransform

### Loading variables 

In [2]:
def load_csv_to_dataframe():
    """Load the loan_payments.csv file and return the resultant dataframe.

    Returns:
        df (pd.Dataframe): dataframe contianing loan_payments data
    """
    
    df = pd.read_csv('../loan_payments.csv')
    return df


In [3]:
# These are used throughout to access the dataframe, and the data_transform object to manipulate the dataframe. 
df = load_csv_to_dataframe()
data_transform = DataTransform(df)
df_info = DataFrameInfo(df)
data_frame_transformer = DataFrameTransform(df)
plotter = Plotter(df)

### Summary statistics 

Calling a few basic functions to get an overview of the dataframe. Calls like .head(), .info, .shape and .describe()

In [4]:
df.head()

,id,member_id,loan_amount,funded_amount,funded_amount_inv,term,int_rate,instalment,grade,sub_grade,...,recoveries,collection_recovery_fee,last_payment_date,last_payment_amount,next_payment_date,last_credit_pull_date,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type
0,38676116,41461848,8000,8000.0,8000.0,36 months,7.49,248.82,A,A4,...,0.0,0.0,Jan-2022,248.82,Feb-2022,Jan-2022,0.0,5.0,1,INDIVIDUAL
1,38656203,41440010,13200,13200.0,13200.0,36 months,6.99,407.52,A,A3,...,0.0,0.0,Jan-2022,407.52,Feb-2022,Jan-2022,0.0,NaN,1,INDIVIDUAL
2,38656154,41439961,16000,16000.0,16000.0,36 months,7.49,497.63,A,A4,...,0.0,0.0,Oct-2021,12850.16,NaN,Oct-2021,0.0,NaN,1,INDIVIDUAL
3,38656128,41439934,15000,15000.0,15000.0,36 months,14.31,514.93,C,C4,...,0.0,0.0,Jun-2021,13899.67,NaN,Jun-2021,0.0,NaN,1,INDIVIDUAL
4,38656121,41439927,15000,15000.0,15000.0,36 months,6.03,456.54,A,A1,...,0.0,0.0,Jan-2022,456.54,Feb-2022,Jan-2022,0.0,NaN,1,INDIVIDUAL


In [5]:
df.shape

(54231, 43)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54231 entries, 0 to 54230
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           54231 non-null  int64  
 1   member_id                    54231 non-null  int64  
 2   loan_amount                  54231 non-null  int64  
 3   funded_amount                51224 non-null  float64
 4   funded_amount_inv            54231 non-null  float64
 5   term                         49459 non-null  object 
 6   int_rate                     49062 non-null  float64
 7   instalment                   54231 non-null  float64
 8   grade                        54231 non-null  object 
 9   sub_grade                    54231 non-null  object 
 10  employment_length            52113 non-null  object 
 11  home_ownership               54231 non-null  object 
 12  annual_inc                   54231 non-null  float64
 13  verification_sta

### Correct the column formats 

##### Dates

In [7]:
# Note how the ouptut format is 'Jan-2021'. we want it to be '2022-01-01'.
# The type of the column is also 'object', and not some kind of date.
df['issue_date'].head()

0    Jan-2021
1    Jan-2021
2    Jan-2021
3    Jan-2021
4    Jan-2021
Name: issue_date, dtype: object

In [8]:
# Column names which are currently objects, but need to be converted to dates 
object_to_date_column_names = ['issue_date', 'last_payment_date', 'next_payment_date', 'last_credit_pull_date']

data_transform.convert_object_columns_to_date(object_to_date_column_names, current_format="%b-%Y")

In [9]:
# The output confirms that the format has changed.
df['last_payment_date'].head()

0   2022-01-01
1   2022-01-01
2   2021-10-01
3   2021-06-01
4   2022-01-01
Name: last_payment_date, dtype: datetime64[ns]

##### Categories 

In [10]:
# Column names which are object types, but are to be converted to categorical types
obj_into_category_col_names = ['grade', 'sub_grade', 'employment_length', 'home_ownership', 'verification_status', 'loan_status', 'purpose', 'earliest_credit_line', 'application_type']

data_transform.convert_object_columns_into_categories(obj_into_category_col_names)


#### Integers 

The `term` column contains only two values: `[36 months, 60 months]`.

I will convert them to integers (since they represent months and could conceivable be a value other than 36 or 60). 

In [11]:
# The following keys are the only two values. Could also simply be made categorical.
term_mapping = {'36 months': 36, '60 months': 60}

# Replace each key with its integer value based on the above map
df['term'] = data_transform.object_to_int(column_name='term', mapping=term_mapping)

In [12]:
# The following keys are the only two values. Could also simply be made categorical.
pay_plan_mapping = {'y': 1, 'n': 0}

# Replace each key with its integer value based on the above map
df['payment_plan'] = data_transform.object_to_int('payment_plan', pay_plan_mapping)

In [13]:
# These four columns all represent a number of months, and so can be cast to int64s
float64_columns_into_int64 = ['collections_12_mths_ex_med', 'mths_since_last_major_derog', 'mths_since_last_delinq', 'mths_since_last_record']

data_transform.convert_float64_columns_into_int64s(float64_columns_into_int64)


### Handle null values

I will first identify the variables with missing values and determine the percentage of missing values in each variable.

In [14]:
# Extract only those columns with null values
columns_with_nulls = df_info.get_columns_with_nulls()

# Create two series objects containing the count and percentage of nulls in those columns 
percent_of_nulls = df_info.percentage_of_nulls_in_data_frame(dataframe=columns_with_nulls).sort_values(ascending=False)
number_of_nulls = df_info.count_nulls_in_data_frame(dataframe=columns_with_nulls)
column_types = columns_with_nulls.dtypes
# Combine that data to display a DataFrame 
data = {
    "% of nulls": percent_of_nulls,
    "# of nulls": number_of_nulls,
    "dtype": column_types
}
both = pd.concat(data, axis=1)
both


,% of nulls,# of nulls,dtype
mths_since_last_record,88.60,48050,Int64
mths_since_last_major_derog,86.17,46732,Int64
next_payment_date,60.13,32608,datetime64[ns]
mths_since_last_delinq,57.17,31002,Int64
int_rate,9.53,5169,float64
term,8.80,4772,float64
funded_amount,5.54,3007,float64
employment_length,3.91,2118,category
last_payment_date,0.13,73,datetime64[ns]
collections_12_mths_ex_med,0.09,51,Int64


In [15]:
df['collections_12_mths_ex_med'].head()

0    0
1    0
2    0
3    0
4    0
Name: collections_12_mths_ex_med, dtype: Int64

#### Dropping columns

**I now will decide which of the above columns are worth dropping entirely, and proceed to do so.**

Based on the above there are 4 columns with very high proportions of nulls:

`[88.60%, 86.17%, 60.13%, 57.17%]`

The next biggest is 9.53%. 

As a result I am going to *remove all columns with >50% null values*.

In [16]:
percent_of_nulls = df_info.percentage_of_nulls_in_data_frame(df)

# Extract columns whose majority (> 50%) of values are null 
more_than_half_null_cols = percent_of_nulls[percent_of_nulls > 50.00]

# Store the names of those columns in a list 
column_names_to_drop = list(more_than_half_null_cols.index)

# Drop the named columns in place from the dataframe 
data_frame_transformer.drop_columns_by_name(column_names_to_drop)

In [17]:
# reset this variable now that the null-heavy columns are removed 
columns_with_nulls = df_info.get_columns_with_nulls()
columns_with_nulls  
# data_frame_transformer.impute_nulls_in_columns(columns_to_impute=columns_with_nulls)



,funded_amount,term,int_rate,employment_length,last_payment_date,last_credit_pull_date,collections_12_mths_ex_med
0,8000.0,36.0,7.49,5 years,2022-01-01,2022-01-01,0
1,13200.0,36.0,6.99,9 years,2022-01-01,2022-01-01,0
2,16000.0,36.0,7.49,8 years,2021-10-01,2021-10-01,0
3,15000.0,36.0,14.31,1 year,2021-06-01,2021-06-01,0
4,15000.0,36.0,6.03,10+ years,2022-01-01,2022-01-01,0
...,...,...,...,...,...,...,...
54226,5000.0,36.0,9.01,1 year,2016-07-01,2016-07-01,<NA>
54227,5000.0,36.0,10.59,< 1 year,2016-10-01,2016-09-01,0
54228,3500.0,36.0,7.43,10+ years,2016-09-01,2013-05-01,<NA>
54229,5000.0,36.0,7.43,4 years,2014-03-01,2013-05-01,<NA>
